In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://security.ubuntu.com

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-29 22:49:17--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.04MB/s    in 0.2s    

2023-01-29 22:49:17 (6.04 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...|2015-08-31 00:00:00|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|      

In [ ]:
# Verifying 0 count for vine column
verfy = df.filter(df.vine == 'Y').count()
verify

0

In [ ]:
# Create the review_analysis dataframe where the total_votes count is equal to or greater than 20
review_analysis = df.filter(df.total_votes >= 20)
review_analysis.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   43836009|R1CY3APKBOVUYU|B00VO8D13K|      82685655|         Citizenfour|Digital_Video_Dow...|          1|            9|         33|   N|                Y|Unnecessarily Slo...|This is an unnece...|2015-08-31 00:00:00|
|         US|   16033254|R2LTI28AN2281A|B00YOSAJBO|     463297585|  Merc

In [ ]:
# new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_votes = review_analysis.filter((review_analysis.helpful_votes / review_analysis.total_votes) >= .5)
helpful_votes.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    6822472| RZM83AGJAUZMA|B0142KIQYA|     273405592|I'll See You in M...|Digital_Video_Dow...|          1|           32|         41|   N|                Y|A Conscious Uncou...|if anyone finds t...|2015-08-31 00:00:00|
|         US|   53001773|R12MYB9QM2YN8B|B00OLI95R6|     955620049|      

In [ ]:
# New DataFrame that retrieves all the rows where a review was written as part of the Vine program (paid)
paid_reviews = helpful_votes.filter(helpful_votes.vine == 'Y')
paid_reviews.show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [ ]:
# Using .count function to get the number of paid reviews
paid_reviews.count()

0

In [ ]:
# Total paid number of 5-star reviews 
paid_five_star = paid_reviews.filter(paid_reviews["star_rating"] == '5').count()
paid_five_star

0

In [ ]:
paid_percentage = (paid_five_star / five_star) * 100
paid_percentage

0.0

In [ ]:
# New DataFrame that retrieves all the rows where a review was written as part of the Vine program (unpaid)
unpaid_reviews = helpful_votes.filter(helpful_votes["vine"] == 'N')
unpaid_reviews.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    6822472| RZM83AGJAUZMA|B0142KIQYA|     273405592|I'll See You in M...|Digital_Video_Dow...|          1|           32|         41|   N|                Y|A Conscious Uncou...|if anyone finds t...|2015-08-31 00:00:00|
|         US|   53001773|R12MYB9QM2YN8B|B00OLI95R6|     955620049|      

In [ ]:
# Using .count function to get the number of unpaid reviews
unpaid_reviews.count()

11371

In [ ]:
# Total unpaid number of 5-star reviews 
unpaid_five_star = unpaid_reviews.filter(unpaid_reviews["star_rating"] == '5').count()
unpaid_five_star

4675

In [ ]:
# Total Number of 5-star reviews 
five_star = helpful_votes.filter(helpful_votes["star_rating"] == '5').count()
five_star

4675

In [ ]:
Unpaid_percentage = (unpaid_five_star / five_star) * 100
Unpaid_percentage

100.0